<img src="images/ragna-logo.png" width="200px" align="right"/>

# RAG and LLM Experiments

<hr>

## Explore the Web UI

Create a ragna.toml file and copy the content from shared/analyst/ragna.toml.tpl file. Update the $USER placeholder in the following fields with your Nebari username:

* `api.root_path`
* `api.url`

Open a terminal window and run the following

1. Activate the conda environment
   
```bash
conda activate global-pycon-de
```

2. Start the UI

```bash
dotenv --file ~/shared/analyst/.env run -- \
    python -m \
        ragna ui --config ragna.toml
```

3. Go to `https://pycon-tutorial.quansight.dev/user/$USER/proxy/31477/` (replace $USER with your Nebari username)

### Aside: Local machine

On your personal computers, you can directly run: `ragna ui` to start the UI and go to `http://localhost:31477` to use it

## Compare LLMs

In [ ]:
import asyncio
import itertools
from pathlib import Path
from pprint import pprint

from dotenv import load_dotenv

from ragna import Rag
from ragna.assistants import Gpt4, Gpt35Turbo16k
from ragna.source_storages import Chroma

from local_llm import Llama38BInstruct

In [ ]:
dotenv_path = Path.home() / Path("shared/analyst/.env")
assert load_dotenv(dotenv_path=dotenv_path)

Let's inquire about PSF's annual reports again.

In [ ]:
documents = [
    "files/psf-report-2021.pdf",
    "files/psf-report-2022.pdf",
]

source_storages = [Chroma]
assistants = [Gpt35Turbo16k, Gpt4, Llama38BInstruct]

prompt = "What was PSF's net income in 2021 and 2022?"

In [ ]:
rag = Rag()

async def answer_prompt(source_storage, assistant):
    async with rag.chat(
        documents=documents,
        source_storage=source_storage,
        assistant=assistant,
    ) as chat:
        message = await chat.answer(prompt)
        return message.content

In [ ]:
experiments = {
    (source_storage.display_name(), assistant.display_name()): answer_prompt(
        source_storage, assistant
    )
    for source_storage, assistant in itertools.product(source_storages, assistants)
}

pprint(experiments)

In [ ]:
results = dict(zip(experiments.keys(), await asyncio.gather(*experiments.values())))
pprint(results)

<hr>

❗️ **Warning:** Make sure to stop the Jupyter Kernel (in the JupyterLab Menu Bar, click on "Kernel" -> "Interrupt Kernel") before proceeding. 

**✨ Next: [Conclusion](05-conclusion.ipynb) →**

<hr>